In [ ]:
!pip install anvil-uplink -qq
!pip install news-please -q
!pip install feedparser -q
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install simpletransformers -qq
!pip install Sastrawi -qq

In [ ]:
from newsplease import NewsPlease
import pandas as pd
import re
import feedparser
from tqdm import tqdm
import anvil.server
import anvil.media
from transformers import pipeline
from transformers import BertTokenizer, EncoderDecoderModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, TFAutoModelForTokenClassification
import string
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
from tqdm.notebook import tqdm
from transformers import pipeline
import numpy as np
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
import torch
print(torch.cuda.is_available())

device = "cuda:0" if torch.cuda.is_available() else "cpu"

try:
  torch.cuda.current_device()
except:
  print("No GPU")

In [ ]:
anvil.server.connect("AIHIM6OIWILFCTPFMT72NV4N-TQP7EZLURCHSQM6M")

# **Crawl**

In [ ]:
    @anvil.server.callable
    def _process(item):
        item = {
            'title': item.get("title", ""),
            'published date': item.get("published", ""),
            'url': item.get("link", ""),
            'publisher': item.get("source", " ")
        }
        return item

    @anvil.server.callable
    def get_news(key):
        if key != '':
            url = key
            return list(map(_process, feedparser.parse(url).entries[:1000]))

In [ ]:
@anvil.server.callable
def t_url(keyword,date_after, date_before):
  return "https://news.google.com/rss/search?hl=id&gl=ID&ceid=ID:id&q="+keyword+"+after:"+"-".join(date_after)+"+before:"+"-".join(date_before)

In [ ]:
@anvil.server.callable
def gen_url_diff(keyword,min_dt, max_dt):
  all_url = []

  # tahun kemarin
  for b in range(12):
    if b<min_dt[1]:
      continue
    for h in range(30):
      if b==min_dt[1] and h<min_dt[2]:
        continue
      #print(x," ",y)
      all_url.append( t_url(keyword, [str(min_dt[0]), str(b), str(h)], [str(min_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(min_dt[0]), str(b), str(31)], ([str(min_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))

  # tahun skrg
  for b in range(12):
    if b>max_dt[1]:
      continue
    for h in range(30):
      if b==max_dt[1] and h>max_dt[2]:
        continue
      all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(h)], [str(max_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(31)], ([str(max_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))


  return all_url

In [ ]:
@anvil.server.callable
def gen_url(keyword,min_dt, max_dt):
  all_url = []

  for b in range(1, 13):
    if b<min_dt[1]:
      continue
    if b>max_dt[1]:
      continue
    for h in range(1, 31):
      if b==min_dt[1] and h<min_dt[2]:
        continue
      if b==max_dt[1] and h>max_dt[2]:
        continue
      all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(h)], [str(max_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(31)], ([str(max_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))
    if max_dt[2] != 31:
      del all_url[-1]
  return all_url

In [ ]:
@anvil.server.callable
def run(keyword,min_date,max_date):
  all_url = gen_url_diff(keyword,min_date,max_date) if max_date[0]!=min_date[0] else gen_url(keyword,min_date,max_date)
  
  result = []
  for url in tqdm(all_url):
    result.append(get_news(url))

  return result

**input**

# **Scrap**

In [ ]:
@anvil.server.callable
def scrap(crawl_df):
  artikel=[]
  for url in tqdm(crawl_df.Link):
    try:
      artikel.append(NewsPlease.from_url(url,timeout=12).maintext)
    except:
      artikel.append(None)
    
  crawl_df.insert(1, "Artikel", artikel, True)

  crawl_df.dropna(inplace=True)
  crawl_df.reset_index(inplace=True)
  del crawl_df['index']
  return crawl_df

In [ ]:
keyword   = "tkbm"
min_date  = [2021,9,1]  # t-b-h
max_date  = [2021,9,10] # t-b-h

#res = run_count(keyword, min_date, max_date)

# **Summarization**

In [ ]:
def remove_outlier(scrap):
  indx_no_tkbm=[]
  for c, (judul, artikel) in enumerate(zip(scrap.Judul, scrap.Artikel)):
    if "tkbm" in judul or "TKBM" in judul or "bongkar muat"  in judul or "Bongkar Muat" in judul\
    or "tkbm"  in artikel or "TKBM"  in artikel or "bongkar muat"  in artikel or "Bongkar Muat" in artikel:
      pass
    else:
      indx_no_tkbm.append(c)

  scrap = scrap.drop(indx_no_tkbm)
  scrap.reset_index(inplace=True)
  del scrap['index']
  return scrap

In [ ]:
@anvil.server.callable
def init_text_summarization(m = "panggi/t5-small-indonesian-summarization-cased"):
  tokenizer = AutoTokenizer.from_pretrained(m)
  model = AutoModelForSeq2SeqLM.from_pretrained(m)
  return tokenizer, model

In [ ]:
#tokenizer, model = init_text_summarization()

In [ ]:
@anvil.server.callable
def get_text_summarization(tokenizer, model, ARTICLE_TO_SUMMARIZE):
  inputs = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt')
  #inputs = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt').to(device)
  #model = model.to(device)
  outputs = model.generate(
      inputs, 
      max_length=150, 
      min_length=40, 
      length_penalty=2.0, 
      repetition_penalty=2.5, 
      num_beams=4,            
      no_repeat_ngram_size=2,
      use_cache=True,
      do_sample = True,
      temperature = 0.8, 
      early_stopping=True,
      top_k = 50,
      top_p = 0.95)

  #summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

# **NER**

In [ ]:
tokenizer_ner = AutoTokenizer.from_pretrained("jplu/tf-xlm-r-ner-40-lang")
model_ner = TFAutoModelForTokenClassification.from_pretrained("jplu/tf-xlm-r-ner-40-lang")
#model_ner = model_ner.to(device)
#nlp_ner = pipeline("ner", model=model_ner, device=0, tokenizer=tokenizer_ner)
nlp_ner = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)

In [ ]:
def get_actor(predictions):
  tmp_per = []
  for key in predictions:
    if key['entity'] == 'PER':
      tmp_per.append(key['word'])
      #print(key['word'])

  PER = [] 
  word_per = ''
  count = 0
  for index, word in enumerate(tmp_per):
    if "▁" in word :
      PER.append(re.sub(r'([▁])', '', word))
      count += 1

    else:
      PER[count-1] += word

    return PER

# **Text Classification**

In [ ]:
pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"
nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    #device=0,
    tokenizer=pretrained_name
)

In [ ]:
pretrained_name_hate = "Hate-speech-CNERG/dehatebert-mono-indonesian"

nlp_hate = pipeline(
    "sentiment-analysis",
    model=pretrained_name_hate,
    tokenizer=pretrained_name_hate
)

# **RUN**

In [ ]:
@anvil.server.callable
def run_count(keyword, min_date, max_date):
  print('KEYWORD ' + keyword)
  print('min date ', min_date)
  print('max date ', max_date)
  crawl = run(keyword.replace(" ", "%20"), min_date, max_date)

  crawl_list=[]
  count=0
  for x in crawl:
    count=count+len(x)
    if x!=[]:
      for key, value in x[0].items():
        temp = [key,value]
        crawl_list.append(temp)
  
  crawl_list=[]
  count=0

  for x in crawl:
    count=count+len(x)
    if x!=[]:
      for y in x:
        temp=[]
        for key, value in y.items():
          temp.append(value)
        crawl_list.append(temp)

  crawl_df = pd.DataFrame(crawl_list)
  crawl_df.columns = ['Judul',"Tanggal","Link","Publisher"]

  # scrap
  print('SCRAP')
  scrap_df = scrap(crawl_df)

  # text sum
  print('SUMMARY')
  #global tokenizer, model
  tokenizer, model = init_text_summarization()
  summary = []
  for teks in tqdm(scrap_df.Artikel):
    summary.append(get_text_summarization(tokenizer, model, teks))

  # ner
  print('NER')
  actor = []
  
  #for teks in tqdm(summary):
  #  try:
  #    predictions = nlp_ner(teks)
  #    actor.append(get_actor(predictions))
  #  except:
  #    actor.append(" ")

  # senti
  print('SENTIMEN')
  sentimen = []
  for teks in tqdm(summary):
    sent = nlp(teks)[0]["label"]
    if sent != "neutral" :
      sentimen.append(sent)
    else:
      hate = nlp_hate(teks)[0]["label"]
      if "NON" in hate:
        sentimen.append("positive")
      elif "NON" not in hate:
        sentimen.append("negative")

  # save
  scrap_df['Summary'] = summary
  #scrap_df['Aktor'] = actor
  scrap_df['Sentimen'] = sentimen

  nama_file = ("scrap_"+keyword)
  scrap_df.to_csv('/tmp/'+nama_file+'.csv')
  scrap_media = anvil.media.from_file('/tmp/'+nama_file+'.csv', 'csv', nama_file)
  print("")
  print("DONE")
  
  return count, scrap_media


#anvil.server.wait_forever()

In [ ]:
run_count("trump", [2021,10,25], [2021,11,2])

In [ ]:
df=pd.read_csv('/tmp/scrap_trump.csv')

In [ ]:
df.to_csv("scrap.csv")